In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584
MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\astro\anaconda3\lib\site-packages\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\astro\anaconda3\lib\site-packages\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype

In [3]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 44s 63ms/step - loss: 0.4036 - acc: 0.8171 - val_loss: 0.3150 - val_acc: 0.8688
Epoch 2/10
625/625 [==============================] - 41s 66ms/step - loss: 0.2387 - acc: 0.9094 - val_loss: 0.2917 - val_acc: 0.8860
Epoch 3/10
625/625 [==============================] - 44s 70ms/step - loss: 0.1827 - acc: 0.9341 - val_loss: 0.2823 - val_acc: 0.8872
Epoch 4/10
625/625 [==============================] - 43s 70ms/step - loss: 0.1502 - acc: 0.9471 - val_loss: 0.4787 - val_acc: 0.8540
Epoch 5/10
625/625 [==============================] - 41s 65ms/step - loss: 0.1313 - acc: 0.9550 - val_loss: 0.3747 - val_acc: 0.8596
Epoch 6/10
625/625 [==============================] - 43s 69ms/step - loss: 0.1111 - acc: 0.9616 - val_loss: 0.3770 - val_acc: 0.8742
Epoch 7/10
625/625 [==============================] - 41s 65ms/step - loss: 0.0962 - acc: 0.9676 - val_loss: 0.3145 - val_acc: 0.8902
Epoch 8/10
625/625 [==============================] - 40s 64ms

In [8]:
results = model.evaluate(test_data, test_labels)

782/782 [==============================] - 12s 15ms/step - loss: 0.4601 - acc: 0.8569


Make Predictions

In [9]:
#this code block convert text to integers for the machine to understand.
word_index = imdb.get_word_index() 
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]
text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [10]:
##this code block convert integers to text for us to understand.
reverse_word_index = {value: key for (key, value) in word_index.items()}
def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "
    return text[:-1]  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [15]:
#predictor
#The more positive a text/review, the higher the number output is. If the number is low, then it's a negative review/text.
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

In [18]:
#To play around, you change add, modify or delete words in a text and watch the output change
positive_review = "That movie was awesome!I really loved it and would great watch it again because it was amazingly great"
predict(positive_review) 

negative_review = "That movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.8935995]
[0.34049943]
